In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os, cv2, h5py
import torch
import torchvision
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import torch.nn.functional as F
import torch.utils.data as utils

from sklearn.metrics import confusion_matrix
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
from time import time
from mpl_toolkits import mplot3d
from torchvision import datasets, transforms
from torch import nn, optim
from torch import autograd

In [3]:
X = h5py.File("./drive/My Drive/data/X_train.h5", "r")["features"][:]
Y = pd.read_csv("./drive/My Drive/data/y_train.csv").as_matrix()[:, 1].squeeze()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [0]:
n = len(X)
x_train = X[:int(0.7*n), 11:]
y_train = Y[:int(0.7*n)]
x_test  = X[int(0.7*n):, 11:]
y_test  = Y[int(0.7*n):]
x_train = x_train[:, ::2]
x_test  = x_test[:, ::2]
x_train = np.array([[j for i, j in enumerate(eeg[10:]) if i % 6] for eeg in x_train])
x_test  = np.array([[j for i, j in enumerate(eeg[10:]) if i % 6] for eeg in x_test])

In [0]:
x_train = np.clip(x_train, -205, 205)
x_test  = np.clip(x_test, -205, 205)

In [0]:
x_train = x_train / 205
x_test  = x_test / 205

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv5x5(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=5, stride=stride,
                     padding=1, bias=False)

def conv7x7(in_planes, out_planes, stride=1):
    return nn.Conv1d(in_planes, out_planes, kernel_size=7, stride=stride,
                     padding=1, bias=False)



class BasicBlock3x3(nn.Module):
    expansion = 1

    def __init__(self, inplanes3, planes, stride=1, downsample=None):
        super(BasicBlock3x3, self).__init__()
        self.conv1 = conv3x3(inplanes3, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class BasicBlock5x5(nn.Module):
    expansion = 1

    def __init__(self, inplanes5, planes, stride=1, downsample=None):
        super(BasicBlock5x5, self).__init__()
        self.conv1 = conv5x5(inplanes5, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv5x5(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        d = residual.shape[2] - out.shape[2]
        out1 = residual[:,:,0:-d] + out
        out1 = self.relu(out1)
        # out += residual

        return out1



class BasicBlock7x7(nn.Module):
    expansion = 1

    def __init__(self, inplanes7, planes, stride=1, downsample=None):
        super(BasicBlock7x7, self).__init__()
        self.conv1 = conv7x7(inplanes7, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv7x7(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        d = residual.shape[2] - out.shape[2]
        out1 = residual[:, :, 0:-d] + out
        out1 = self.relu(out1)
        # out += residual

        return out1




class MSResNet(nn.Module):
    def __init__(self, input_channel, layers=[1, 1, 1, 1], num_classes=10):
        self.inplanes3 = 64
        self.inplanes5 = 64
        self.inplanes7 = 64

        super(MSResNet, self).__init__()

        self.conv1 = nn.Conv1d(input_channel, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer3x3_1 = self._make_layer3(BasicBlock3x3, 64, layers[0], stride=2)
        self.layer3x3_2 = self._make_layer3(BasicBlock3x3, 128, layers[1], stride=2)
        self.layer3x3_3 = self._make_layer3(BasicBlock3x3, 256, layers[2], stride=2)
        # self.layer3x3_4 = self._make_layer3(BasicBlock3x3, 512, layers[3], stride=2)

        # maxplooing kernel size: 16, 11, 6
        self.maxpool3 = nn.AvgPool1d(kernel_size=16, stride=1, padding=0)


        self.layer5x5_1 = self._make_layer5(BasicBlock5x5, 64, layers[0], stride=2)
        self.layer5x5_2 = self._make_layer5(BasicBlock5x5, 128, layers[1], stride=2)
        self.layer5x5_3 = self._make_layer5(BasicBlock5x5, 256, layers[2], stride=2)
        # self.layer5x5_4 = self._make_layer5(BasicBlock5x5, 512, layers[3], stride=2)
        self.maxpool5 = nn.AvgPool1d(kernel_size=11, stride=1, padding=0)


        self.layer7x7_1 = self._make_layer7(BasicBlock7x7, 64, layers[0], stride=2)
        self.layer7x7_2 = self._make_layer7(BasicBlock7x7, 128, layers[1], stride=2)
        self.layer7x7_3 = self._make_layer7(BasicBlock7x7, 256, layers[2], stride=2)
        # self.layer7x7_4 = self._make_layer7(BasicBlock7x7, 512, layers[3], stride=2)
        self.maxpool7 = nn.AvgPool1d(kernel_size=6, stride=1, padding=0)

        # self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(256*3, 128)

        # todo: modify the initialization
        # for m in self.modules():
        #     if isinstance(m, nn.Conv1d):
        #         n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
        #         m.weight.data.normal_(0, math.sqrt(2. / n))
        #     elif isinstance(m, nn.BatchNorm1d):
        #         m.weight.data.fill_(1)
        #         m.bias.data.zero_()

    def _make_layer3(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes3 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes3, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes3, planes, stride, downsample))
        self.inplanes3 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes3, planes))

        return nn.Sequential(*layers)

    def _make_layer5(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes5 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes5, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes5, planes, stride, downsample))
        self.inplanes5 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes5, planes))

        return nn.Sequential(*layers)


    def _make_layer7(self, block, planes, blocks, stride=2):
        downsample = None
        if stride != 1 or self.inplanes7 != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes7, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes7, planes, stride, downsample))
        self.inplanes7 = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes7, planes))

        return nn.Sequential(*layers)

    def forward(self, x0):
        x0 = self.conv1(x0)
        x0 = self.bn1(x0)
        x0 = self.relu(x0)
        x0 = self.maxpool(x0)
        x = self.layer3x3_1(x0)
        x = self.layer3x3_2(x)
        x = self.layer3x3_3(x)
        # x = self.layer3x3_4(x)
        x = self.maxpool3(x)

        y = self.layer5x5_1(x0)
        y = self.layer5x5_2(y)
        y = self.layer5x5_3(y)
        # y = self.layer5x5_4(y)
        y = self.maxpool5(y)

        z = self.layer7x7_1(x0)
        z = self.layer7x7_2(z)
        z = self.layer7x7_3(z)
        # z = self.layer7x7_4(z)
        z = self.maxpool7(z)

        out = torch.cat([x, y, z], dim=1)
        
        out = out.squeeze()
        # out = self.drop(out)
        out1 = self.fc(out)

        return out1

In [0]:
def get_distance(data):
    dot_product = torch.matmul(data, data.transpose(1, 0))
    diag        = torch.diag(dot_product)
    distances   = diag.view((1, diag.shape[0])) - 2*dot_product + diag.view((diag.shape[0], 1))
    distances   = torch.max(distances, torch.tensor([0]).float().cuda())
    mask        = torch.eq(distances, torch.zeros(1).cuda()).float().cuda()
    distances   = distances + mask * 1e-16
    distances   = torch.sqrt(distances)
    distances   = distances * (1.0 - mask)
    return distances

In [0]:
def get_mask_triplet(labels):
    hot = F.one_hot(labels, 3).float()
    positive_mask = torch.matmul(hot, hot.transpose(0, 1))
    size = positive_mask.shape[0]
    positive_mask[range(size), range(size)] = torch.zeros(positive_mask.shape[0]).long().cuda()
    negative_mask = torch.matmul(torch.ones(hot.shape).long().cuda() - hot, hot.transpose(0, 1))
    mask = torch.matmul(positive_mask.view((size, size, 1)), negative_mask.view((size, 1, size)))
    return mask

In [0]:
def triplet_loss(data, labels, m):
    distances = get_distance(data)
    size = distances.shape[0]
    delta_p   = distances.view((size, size, 1)).repeat((1, 1, size)) + m
    delta_n   = distances.view((size, 1, size)).repeat((1, size, 1))
    triplet_loss = delta_p - delta_n
    mask_hard = torch.gt(delta_p, delta_n).float()
    mask      = get_mask_triplet(labels).float() * mask_hard
    triplet_loss = triplet_loss * mask
    triplet_loss = torch.max(triplet_loss, torch.zeros(1).float().cuda())
    
    nb_of_tensor = torch.gt(triplet_loss, torch.tensor([1e-16]).float().cuda())
    nb_of_tensor = torch.sum(nb_of_tensor)
    triplet_loss = torch.sum(triplet_loss)
    return triplet_loss / (nb_of_tensor + torch.tensor([1e-16]).float().cuda())

In [0]:
def augment(serie):
    noise = torch.randn(serie.shape[0], 1, serie.shape[-1]).float().cuda() * 0.005
    serie = noise + serie
    coins = torch.gt(torch.randn((serie.shape[0], 1, 1)).float().cuda(), torch.tensor([0]).float().cuda()).float().cuda() * 2 - 1
    serie = serie * coins
    return serie

In [0]:
x_train = torch.tensor(x_train).float().cuda()
x_test  = torch.tensor(x_test).float().cuda()
y_train = torch.tensor(y_train).long().cuda()
y_test  = torch.tensor(y_test).long().cuda()

In [0]:
x_train = x_train.view((-1, 1, 512))
x_test  = x_test.view((-1, 1, 512))

In [0]:
train_data = utils.TensorDataset(x_train, y_train)
test_data  = utils.TensorDataset(x_test, y_test)

In [0]:
train_loader = utils.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = utils.DataLoader(test_data, batch_size=64, shuffle=True)

In [0]:
model = MSResNet(1).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.009)

In [56]:
epochs = 100
best_loss = 1000
for e in range(epochs):
    running_loss = 0
    test_loss    = 0
    for x, y in train_loader:
        x = augment(x)
        optimizer.zero_grad()
        output = model(x)
        loss   = triplet_loss(output, y, torch.tensor([1]).float().cuda())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print('Current loss : {0:.2f}'.format(loss.item()))
    else:
      for x, y in test_loader:
        out  = model(x)
        loss = loss_function(out, y)
        test_loss += loss.item()
      test_loss /= len(test_loader)

      if test_loss < best_loss:
        torch.save(model, './drive/My Drive/data/model.pt')
        best_loss  = test_loss
      print('Test loss : {}, Best loss : {}'.format(test_loss, best_loss))

Current loss : 1.05
Current loss : 1.07
Current loss : 1.08
Current loss : 1.11
Current loss : 1.14
Current loss : 1.01
Current loss : 0.94
Current loss : 1.05
Current loss : 1.06
Current loss : 1.12
Current loss : 1.12
Current loss : 1.19
Current loss : 1.04
Current loss : 1.11
Current loss : 0.94
Current loss : 1.16
Current loss : 1.08
Current loss : 1.09
Current loss : 0.94
Current loss : 1.07
Current loss : 1.15
Current loss : 1.06
Current loss : 1.15
Current loss : 1.07
Current loss : 1.04
Current loss : 1.05
Current loss : 1.13
Current loss : 1.09
Current loss : 1.00
Current loss : 1.11
Current loss : 1.17
Current loss : 1.03
Current loss : 1.08
Current loss : 1.01
Current loss : 1.13
Current loss : 1.03
Current loss : 1.06
Current loss : 1.11
Current loss : 1.06
Current loss : 1.00
Current loss : 1.04
Current loss : 1.01
Current loss : 0.97
Current loss : 1.02
Current loss : 1.00
Current loss : 1.03
Current loss : 1.05
Current loss : 1.01
Current loss : 1.07
Current loss : 0.94


KeyboardInterrupt: ignored

In [0]:
test = torch.from_numpy(test).view(1, 1, 512).float()

In [0]:
cc = model(test)